## Imports

In [7]:
import re
import requests

import pandas as pd
from tqdm import tqdm


from sentence_transformers import SentenceTransformer

import chromadb

# Только для юпитер-ноутбука
import sys
import os
sys.path.append(os.path.abspath('../'))
# -----

from src.metrics.geval import LLMTestCase, relevance_metric, answer_relevance_metric, hallucination_detection_metric
from src.metrics.recall import calculate_recall
from src.metrics.ndcg import calculate_ndcg
tqdm.pandas()

In [8]:
DEVICE = 'cuda'
CHROMA_DB_PATH = '../data/chroma_db'
DATA_FOR_DB_PARQUET = '../data/actual/interim/data_for_db_corrected.parquet'
EVALUATION_PARQUET = '../data/actual/interim/evaluation_df.parquet'
SMALL_EVALUATION_PARQUET = '../data/actual/interim/evaluation_df_125.parquet'
CLASSIFICATOR_ENDPOINT = 'http://localhost:8000/classify'
EMBEDDER = 'intfloat/multilingual-e5-large'
# EMBEDDER = '../e5_finetuned'

## Data for db

In [9]:
data_for_db = pd.read_parquet(DATA_FOR_DB_PARQUET)

In [10]:
data_for_db.head(1)

,index,kb_query,kb_answer,class_1,class_2,kb_query_orig
0,0,Что нельзя публиковать на RUTUBE (рутуб)?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео,Что нельзя публиковать на RUTUBE?


In [11]:
data_for_db = data_for_db.fillna('')

## Embedder

In [12]:
embedder = SentenceTransformer(EMBEDDER,device=DEVICE)

# Обратите внимание на prompt 
#  query:
#  passage:

# input_texts = [
#     'query: how much protein should a female eat',
#     'query: 南瓜的家常做法',
#     "passage: As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 i     s 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or traini     ng for a marathon. Check out the chart below to see how much protein you should be eating each day.",
#     "passage: 1.清炒南瓜丝 原料:嫩南瓜半个 调料:葱、盐、白糖、鸡精 做法: 1、南瓜用刀薄薄的削去表面一层皮     ,用勺子刮去瓤 2、擦成细丝(没有擦菜板就用刀慢慢切成细丝) 3、锅烧热放油,入葱花煸出香味 4、入南瓜丝快速翻炒一分钟左右,     放盐、一点白糖和鸡精调味出锅 2.香葱炒南瓜 原料:南瓜1只 调料:香葱、蒜末、橄榄油、盐 做法: 1、将南瓜去皮,切成片 2、油     锅8成热后,将蒜末放入爆香 3、爆香后,将南瓜片放入,翻炒 4、在翻炒的同时,可以不时地往锅里加水,但不要太多 5、放入盐,炒匀      6、南瓜差不多软和绵了之后,就可以关火 7、撒入香葱,即可出锅"
# ]
# embeddings = embedder.encode(input_texts, normalize_embeddings=True)

Here are some rules of thumb:

Use "query: " and "passage: " correspondingly for asymmetric tasks such as passage retrieval in open QA, ad-hoc information retrieval.

Use "query: " prefix for symmetric tasks such as semantic similarity, bitext mining, paraphrase retrieval.

Use "query: " prefix if you want to use embeddings as features, such as linear probing classification, clustering.

## Database

In [13]:
client = chromadb.PersistentClient(path=CHROMA_DB_PATH, settings=chromadb.Settings(allow_reset=True))

In [14]:
client.reset()

True

In [15]:
collection = client.get_or_create_collection(name="rutube",metadata={"hnsw:space": "cosine"})

In [16]:
documents = data_for_db['kb_query'].tolist()

In [17]:
documents_query = ['query: ' + doc for doc in documents]
embeddings = embedder.encode(documents_query, show_progress_bar=True)

Batches:   0%|          | 0/20 [00:00<?, ?it/s]

In [18]:
metadatas = data_for_db.drop(columns=['index','kb_query']).to_dict(orient='index')
metadatas = [data[1] for n,data in enumerate(metadatas.items())]

In [19]:
ids = [str(x) for x in data_for_db['index'].tolist()]

In [20]:
collection.add(
    documents=documents,
    embeddings=embeddings,
    metadatas=metadatas,
    ids=ids
)

## Classificator

In [21]:
def classify(question:str)->str:
    response = requests.post(CLASSIFICATOR_ENDPOINT,json={'question':question})
    return response.json()['class_1'],response.json()['class_2']

## Retriever

In [22]:
def retrieve_relevant_chunks(query:str | list[str], where: dict | None = None, where_document: dict | None = None, n_results:int = 3) -> None:
    '''
    where={"metadata_field": "is_equal_to_this"},
    where_document={"$contains":"search_string"}
    '''

    if isinstance(query,str):
        query=[query]

    query = ['query: ' + q for q in query]
        
    query_embeddings = embedder.encode(query)

    query_kwargs = {
        'query_embeddings': query_embeddings,
        'include': ['documents','metadatas'],
        'n_results': n_results
    }
    if where:
        query_kwargs['where']=where

    if where_document:
        query_kwargs['where_document'] = where_document

    result = collection.query(**query_kwargs)
    return result

def format_retrieve_result(retrieve_result) -> list:
    result = []

    ids = retrieve_result.get('ids', [])
    documents = retrieve_result.get('documents', [])
    metadatas = retrieve_result.get('metadatas', [])

    for i in range(len(ids)):
        group = []
        for j in range(len(ids[i])):
            entry = {
                'id': ids[i][j],
                'document': documents[i][j],
                'metadata': metadatas[i][j] if metadatas else None
            }
            group.append(entry)
        result.append(group)

    return result

def extract_context_for_generation(retrieved_result: list[dict], include_question: bool = True) -> str:
    if retrieved_result:
        result_string = "\n\n".join(
            [
                (f"Вопрос: {item['document']}\nОтвет: {item['metadata']['kb_answer']}" if include_question else item['metadata']['kb_answer'])
                for sublist in retrieved_result for item in sublist
            ]
        )
    else:
        result_string = ''
    
    return result_string

## Generation

In [23]:
def generate(prompt:str, temperature:float=0, return_str_only:bool=True) -> dict:
    result = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": "mistral-nemo",
            "options": {"seed": 123, "temperature": temperature},
            "prompt": prompt,
            "stream": False,
        },
    ).json()
    if return_str_only:
        return result['response']
    else:
        return result

## Chunk LLM filtering prompt (optional)

In [24]:
FILTERING_PROMPT = '''Отвечает ли на вопрос предоставленная инструкция?

Ответь 0 или 1 и больше ничего.
0 если не отвечает.
1 если отвечает.

ВОПРОС:
=====
{query}
=====

ИНСТРУКЦИЯ:
=====
{context}
=====
'''.strip()

In [25]:
def chunk_filtering(query:str,context:str) -> bool:
    response = generate(FILTERING_PROMPT.format(query=query, context=context))
    if '0' in response[:10]:
        return False
    elif '1' in response[:10]:
        return True

## Prompt template

In [26]:
PROMPT_TEMPLATE = '''
Ты интеллектуальный помощник службы поддержки RUTUBE. 
RUTUBE — ведущий российский видеопортал, предлагающий к просмотру тв онлайн, кинофильмы, сериалы, мультфильмы и пользовательское видео.
Ты очень вежливый и дружелюбный. Твоя задача ответить на ВОПРОС используя КОНТЕКСТ. 

У тебя есть только знания из контекста и этого промпта.
Если ответа на вопрос к контексте нет, то напиши, что не знаешь ответа.
Отвечай только используя контекст, не используй другие знания.

ВОПРОС:
=====
{query}
=====

КОНТЕКСТ:
=====
{context}
=====
'''.strip()

## Complete QA-RAG pipeline

In [27]:
def rag_pipeline(input:str) -> dict:
    # class_1 = classify(input)[0]

    actual_context_raw = retrieve_relevant_chunks(
        input,
        n_results=10, 
        # where={'class_1':class_1}
        )
    actual_context = format_retrieve_result(actual_context_raw)

    # context_llm_filtered = []
    # for cont in actual_context[0]: # КОСТЫЛЬ
    #     if chunk_filtering(input,extract_context_for_generation([[cont]])):
    #         context_llm_filtered.append(cont)

    # context_llm_filtered = [context_llm_filtered]    

    # actual_context_to_llm = extract_context_for_generation(context_llm_filtered)
    actual_context_to_llm = extract_context_for_generation(actual_context)
    prompt = PROMPT_TEMPLATE.format(query=input,context=actual_context_to_llm)
    response = generate(prompt)


    result = {
        'input':input,
        'actual_context': actual_context,
        'actual_response': response
    }

    return result

rag_pipeline('Как включить монетизацию?')

{'input': 'Как включить монетизацию?',
 'actual_context': [[{'id': '173',
    'document': 'Как подключить монетизацию, если я физлицо?',
    'metadata': {'class_1': 'ПРЕДЛОЖЕНИЯ',
     'class_2': 'Монетизация',
     'kb_answer': 'Увы, физлицам монетизация недоступна, зато доступна самозанятым. Стать самозанятым можно за пару минут через приложение «Мой налог».  ',
     'kb_query_orig': 'Как подключить монетизацию, если я физлицо?'}},
   {'id': '189',
    'document': 'Как формируется вознаграждение за монетизацию?',
    'metadata': {'class_1': 'МОНЕТИЗАЦИЯ',
     'class_2': 'Статистика по монетизации',
     'kb_answer': 'Для расчета вознаграждения партнера используется специальная формула, указанная в п.3.1. Оферты. Актуальные показатели формирующие общее итоговое вознаграждение партнера также отображается в Студии RUTUBE (студии рутуб) в разделе «Монетизация»: https://studio.rutube.ru/monetization',
     'kb_query_orig': 'Как формируется вознаграждение за монетизацию?'}},
   {'id': '17

## Evaluation

In [28]:
eval_df = pd.read_parquet(EVALUATION_PARQUET)

In [29]:
eval_df.head(2)

,input,expected_output,kb_query,index,kb_query_orig
0,Здравствуйте! Можно уточнить причины Правилhtt...,Добрый день!\nЧто нельзя публиковать на RUTUBE...,Что нельзя публиковать на RUTUBE?,0.0,Что нельзя публиковать на RUTUBE?
1,"Добрый вечер, какой топ причин блокировки виде...",Добрый вечер!\nЧто заперщено публиковать на RU...,Что нельзя публиковать на RUTUBE?,0.0,Что нельзя публиковать на RUTUBE?


In [30]:
# def stratified_sample(df, stratify_column, frac):
#     return df.groupby(stratify_column, group_keys=False).apply(lambda x: x.sample(frac=frac))

# stratified_df = stratified_sample(eval_df, stratify_column='index', frac=0.2)


In [31]:
stratified_df = pd.read_parquet(SMALL_EVALUATION_PARQUET)

In [ ]:
def process_row(row):
    result = rag_pipeline(row['input'])
    
    actual_response = result.get('actual_response', '')

    actual_context_ids = [item['id'] for sublist in result['actual_context'] for item in sublist]

    class_1_list = [item['metadata']['class_1'] for sublist in result['actual_context'] for item in sublist]
    class_2_list = [item['metadata']['class_2'] for sublist in result['actual_context'] for item in sublist]

    actual_context = extract_context_for_generation(result['actual_context'])

    return pd.Series({
        'actual_response': actual_response,
        'actual_context_ids': actual_context_ids,
        'actual_context': actual_context,
        'class_1': class_1_list,
        'class_2': class_2_list
    })

stratified_df[['actual_response', 'actual_context_ids', 'actual_context', 'class_1', 'class_2']] = stratified_df.progress_apply(process_row, axis=1)


In [27]:
stratified_df

,input,expected_output,kb_query,index,kb_query_orig,actual_response,actual_context_ids,actual_context,class_1,class_2
2,"Все пишут, что монетизация на рутубе отключает...","Добрый день! \nМонетизация может отключиться, ...",Почему могут отключить монетизацию из-за автор...,1.0,Почему могут отключить монетизацию из-за автор...,"Вопрос: Как я могу узнать, какие выплаты я дол...","[410, 1, 408, 531, 170, 421, 182, 466, 2, 178]",Вопрос: Как проходит процесс интеграции моего ...,"[МОНЕТИЗАЦИЯ, МОНЕТИЗАЦИЯ, МОНЕТИЗАЦИЯ, МОНЕТИ...","[Монетизация, Отключение/подключение монетизац..."
9,"Подскажите, пожалуйста, что не запрещено выкла...","Добрый день! Не запрешено публиковать то, что ...",Какой контент можно использовать для монетизац...,4.0,Какой контент можно использовать для монетизац...,"Вопрос:\n=====\nПодскажите, пожалуйста, что та...","[3, 4, 1, 6, 2, 184, 174, 167, 178, 123]",Вопрос: Для каких статусов доступна монетизаци...,"[МОНЕТИЗАЦИЯ, МОНЕТИЗАЦИЯ, МОНЕТИЗАЦИЯ, МОНЕТИ...","[Отключение/подключение монетизации, Отключени..."
18,Кому принадлежат авторские права на данный кон...,"По умолчанию считается, что авторские права на...","Кому принадлежат авторские права на контент, к...",7.0,"Кому принадлежат авторские права на контент, к...","Авторские права на контент, который вы размеща...","[7, 5, 626, 591, 424, 414, 450, 587, 594, 603]",Вопрос: Кому принадлежат авторские права на ко...,"[МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, М...","[Отклонение/блокировка видео, Отклонение/блоки..."
19,"пройдет видео модерация, если цитировать чужие...",Для прохождения модерации нужно соблюдать неск...,Можно ли цитировать чужие произведения?,8.0,Можно ли цитировать чужие произведения?,Если вы цитируете чужие материалы в своем виде...,"[8, 12, 11, 15, 418, 424, 610, 199, 474, 587]",Вопрос: Можно ли цитировать чужие произведения...,"[МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, М...","[Отклонение/блокировка видео, Отклонение/блоки..."
32,произведние в общественном достоянии? О чем это?,"Это значит, что произведение или его перевод п...",Нарушает ли авторские права использование обра...,13.0,Нарушает ли авторские права использование обра...,В контексте нет информации о произведении в об...,"[423, 614, 67, 434, 496, 299, 312, 567, 57, 58]",Вопрос: У меня возник вопрос: каковы последств...,"[УПРАВЛЕНИЕ АККАУНТОМ, УПРАВЛЕНИЕ АККАУНТОМ, У...","[Студия RUTUBE, Верификация, Персонализация, О..."
...,...,...,...,...,...,...,...,...,...,...
733,Какое видео могу загрузить?,Оно должно соответствовать правилам размещения...,Какие технические требования предъявляются к з...,272.0,Какие технические требования предъявляются к з...,"На основании контекста, можно сделать вывод, ч...","[236, 264, 278, 263, 87, 276, 245, 518, 242, 238]",Вопрос: Как загрузить видеоролик на сайт?\nОтв...,"[ВИДЕО, ВИДЕО, ВИДЕО, ВИДЕО, ВИДЕО, ВИДЕО, ВИД...","[Загрузка видео, Загрузка видео, Загрузка виде..."
684,Почнм нельзя смотреть видео в хорошем качестве?,"Вероятнее всего, у вас стоит блокировщик рекла...",Не могу посмотреть ролик в хорошем качестве,277.0,Не могу посмотреть ролик в хорошем качестве,Почему нельзя смотреть видео в хорошем качеств...,"[242, 241, 278, 264, 280, 92, 237, 156, 239, 279]",Вопрос: Не могу посмотреть ролик в хорошем кач...,"[ВИДЕО, ВИДЕО, ВИДЕО, ВИДЕО, ВИДЕО, ВИДЕО, ВИД...","[Воспроизведение видео, Воспроизведение видео,..."
690,Как правильно выбрать категорию?,На RUTUBE доступно большое количество категори...,Какую категорию выбрать для видео?,280.0,Какую категорию выбрать для видео?,"Насколько я понимаю, вы предоставили мне инфор...","[245, 90, 158, 254, 85, 278, 250, 88, 155, 92]",Вопрос: Какую категорию выбрать для видео?\nОт...,"[ВИДЕО, ВИДЕО, ВИДЕО, ВИДЕО, ВИДЕО, ВИДЕО, ВИД...","[Загрузка видео, Загрузка видео, Загрузка виде..."
696,У вас встроенные субтитры или сгенерированые?,"Это могут быть субтитры, которые добавил сам а...",Какие субтитры используются в RUTUBE?,283.0,Какие субтитры используются в RUTUBE?,"У нас есть как встроенные, так и сгенер

## Retrieve recall

In [28]:
stratified_df['hit_list'] = stratified_df.apply(
    lambda x: [int(int(y) == int(x['index'])) for y in x['actual_context_ids']],
    axis=1
)


In [29]:
stratified_df['recall'] = stratified_df.apply(
    lambda x: sum(x['hit_list']) / 1 if len(x['actual_context_ids']) > 0 else 0,
    axis=1
) # WATCHOUT: везде len(true_context) == 1


In [30]:
stratified_df

,input,expected_output,kb_query,index,kb_query_orig,actual_response,actual_context_ids,actual_context,class_1,class_2,hit_list,recall
2,"Все пишут, что монетизация на рутубе отключает...","Добрый день! \nМонетизация может отключиться, ...",Почему могут отключить монетизацию из-за автор...,1.0,Почему могут отключить монетизацию из-за автор...,"Вопрос: Как я могу узнать, какие выплаты я дол...","[410, 1, 408, 531, 170, 421, 182, 466, 2, 178]",Вопрос: Как проходит процесс интеграции моего ...,"[МОНЕТИЗАЦИЯ, МОНЕТИЗАЦИЯ, МОНЕТИЗАЦИЯ, МОНЕТИ...","[Монетизация, Отключение/подключение монетизац...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",1.0
9,"Подскажите, пожалуйста, что не запрещено выкла...","Добрый день! Не запрешено публиковать то, что ...",Какой контент можно использовать для монетизац...,4.0,Какой контент можно использовать для монетизац...,"Вопрос:\n=====\nПодскажите, пожалуйста, что та...","[3, 4, 1, 6, 2, 184, 174, 167, 178, 123]",Вопрос: Для каких статусов доступна монетизаци...,"[МОНЕТИЗАЦИЯ, МОНЕТИЗАЦИЯ, МОНЕТИЗАЦИЯ, МОНЕТИ...","[Отключение/подключение монетизации, Отключени...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",1.0
18,Кому принадлежат авторские права на данный кон...,"По умолчанию считается, что авторские права на...","Кому принадлежат авторские права на контент, к...",7.0,"Кому принадлежат авторские права на контент, к...","Авторские права на контент, который вы размеща...","[7, 5, 626, 591, 424, 414, 450, 587, 594, 603]",Вопрос: Кому принадлежат авторские права на ко...,"[МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, М...","[Отклонение/блокировка видео, Отклонение/блоки...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1.0
19,"пройдет видео модерация, если цитировать чужие...",Для прохождения модерации нужно соблюдать неск...,Можно ли цитировать чужие произведения?,8.0,Можно ли цитировать чужие произведения?,Если вы цитируете чужие материалы в своем виде...,"[8, 12, 11, 15, 418, 424, 610, 199, 474, 587]",Вопрос: Можно ли цитировать чужие произведения...,"[МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, М...","[Отклонение/блокировка видео, Отклонение/блоки...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1.0
32,произведние в общественном достоянии? О чем это?,"Это значит, что произведение или его перевод п...",Нарушает ли авторские права использование обра...,13.0,Нарушает ли авторские права использование обра...,В контексте нет информации о произведении в об...,"[423, 614, 67, 434, 496, 299, 312, 567, 57, 58]",Вопрос: У меня возник вопрос: каковы последств...,"[УПРАВЛЕНИЕ АККАУНТОМ, УПРАВЛЕНИЕ АККАУНТОМ, У...","[Студия RUTUBE, Верификация, Персонализация, О...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
733,Какое видео могу загрузить?,Оно должно соответствовать правилам размещения...,Какие технические требования предъявляются к з...,272.0,Какие технические требования предъявляются к з...,"На основании контекста, можно сделать вывод, ч...","[236, 264, 278, 263, 87, 276, 245, 518, 242, 238]",Вопрос: Как загрузить видеоролик на сайт?\nОтв...,"[ВИДЕО, ВИДЕО, ВИДЕО, ВИДЕО, ВИДЕО, ВИДЕО, ВИД...","[Загрузка видео, Загрузка видео, Загрузка виде...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.0
684,Почнм нельзя смотреть видео в хорошем качестве?,"Вероятнее всего, у вас стоит блокировщик рекла...",Не могу посмотреть ролик в хорошем качестве,277.0,Не могу посмотреть ролик в хорошем качестве,Почему нельзя смотреть видео в хорошем качеств...,"[242, 241, 278, 264, 280, 92, 237, 156, 239, 279]",Вопрос: Не могу посмотреть ролик в хорошем кач...,"[ВИДЕО, ВИДЕО, ВИДЕО, ВИДЕО, ВИДЕО, ВИДЕО, ВИД...","[Воспроизведение видео, Воспроизведение видео,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.0
690,Как правильно выбрать категорию?,На RUTUBE доступно большое количество категори...,Какую категорию выбрать для видео?,280.0,Какую категорию выбрать для видео?,"Насколько я понимаю, вы предоставили мне инфор...","[245, 90, 158, 254, 85, 278, 250, 88, 155, 92]",Вопрос: Какую категорию выбрать для видео?\nОт...,"[ВИДЕО, ВИДЕО, ВИДЕО, ВИДЕО, ВИДЕО, ВИДЕО, В

In [31]:
stratified_df['recall'].mean()

0.2

0.208

0.248

0.168

0.24

0.2

## Retrieve NDCG

In [32]:
stratified_df['hit_list']

2      [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
9      [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
18     [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
19     [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
32     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
                    ...              
733    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
684    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
690    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
696    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
702    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Name: hit_list, Length: 125, dtype: object

In [33]:
stratified_df['ndcg'] = stratified_df['hit_list'].apply(lambda x: calculate_ndcg([x])[0]) # КОСТЫЛЬ

In [34]:
stratified_df

,input,expected_output,kb_query,index,kb_query_orig,actual_response,actual_context_ids,actual_context,class_1,class_2,hit_list,recall,ndcg
2,"Все пишут, что монетизация на рутубе отключает...","Добрый день! \nМонетизация может отключиться, ...",Почему могут отключить монетизацию из-за автор...,1.0,Почему могут отключить монетизацию из-за автор...,"Вопрос: Как я могу узнать, какие выплаты я дол...","[410, 1, 408, 531, 170, 421, 182, 466, 2, 178]",Вопрос: Как проходит процесс интеграции моего ...,"[МОНЕТИЗАЦИЯ, МОНЕТИЗАЦИЯ, МОНЕТИЗАЦИЯ, МОНЕТИ...","[Монетизация, Отключение/подключение монетизац...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",1.0,0.63093
9,"Подскажите, пожалуйста, что не запрещено выкла...","Добрый день! Не запрешено публиковать то, что ...",Какой контент можно использовать для монетизац...,4.0,Какой контент можно использовать для монетизац...,"Вопрос:\n=====\nПодскажите, пожалуйста, что та...","[3, 4, 1, 6, 2, 184, 174, 167, 178, 123]",Вопрос: Для каких статусов доступна монетизаци...,"[МОНЕТИЗАЦИЯ, МОНЕТИЗАЦИЯ, МОНЕТИЗАЦИЯ, МОНЕТИ...","[Отключение/подключение монетизации, Отключени...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",1.0,0.63093
18,Кому принадлежат авторские права на данный кон...,"По умолчанию считается, что авторские права на...","Кому принадлежат авторские права на контент, к...",7.0,"Кому принадлежат авторские права на контент, к...","Авторские права на контент, который вы размеща...","[7, 5, 626, 591, 424, 414, 450, 587, 594, 603]",Вопрос: Кому принадлежат авторские права на ко...,"[МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, М...","[Отклонение/блокировка видео, Отклонение/блоки...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1.0,1.00000
19,"пройдет видео модерация, если цитировать чужие...",Для прохождения модерации нужно соблюдать неск...,Можно ли цитировать чужие произведения?,8.0,Можно ли цитировать чужие произведения?,Если вы цитируете чужие материалы в своем виде...,"[8, 12, 11, 15, 418, 424, 610, 199, 474, 587]",Вопрос: Можно ли цитировать чужие произведения...,"[МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, М...","[Отклонение/блокировка видео, Отклонение/блоки...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1.0,1.00000
32,произведние в общественном достоянии? О чем это?,"Это значит, что произведение или его перевод п...",Нарушает ли авторские права использование обра...,13.0,Нарушает ли авторские права использование обра...,В контексте нет информации о произведении в об...,"[423, 614, 67, 434, 496, 299, 312, 567, 57, 58]",Вопрос: У меня возник вопрос: каковы последств...,"[УПРАВЛЕНИЕ АККАУНТОМ, УПРАВЛЕНИЕ АККАУНТОМ, У...","[Студия RUTUBE, Верификация, Персонализация, О...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
733,Какое видео могу загрузить?,Оно должно соответствовать правилам размещения...,Какие технические требования предъявляются к з...,272.0,Какие технические требования предъявляются к з...,"На основании контекста, можно сделать вывод, ч...","[236, 264, 278, 263, 87, 276, 245, 518, 242, 238]",Вопрос: Как загрузить видеоролик на сайт?\nОтв...,"[ВИДЕО, ВИДЕО, ВИДЕО, ВИДЕО, ВИДЕО, ВИДЕО, ВИД...","[Загрузка видео, Загрузка видео, Загрузка виде...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.0,0.00000
684,Почнм нельзя смотреть видео в хорошем качестве?,"Вероятнее всего, у вас стоит блокировщик рекла...",Не могу посмотреть ролик в хорошем качестве,277.0,Не могу посмотреть ролик в хорошем качестве,Почему нельзя смотреть видео в хорошем качеств...,"[242, 241, 278, 264, 280, 92, 237, 156, 239, 279]",Вопрос: Не могу посмотреть ролик в хорошем кач...,"[ВИДЕО, ВИДЕО, ВИДЕО, ВИДЕО, ВИДЕО, ВИДЕО, ВИД...","[Воспроизведение видео, Воспроизведение видео,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.0,0.00000
690,Как правильно выбрать категорию?,На RUTUBE доступно большое количество категори...,Какую категорию выбрать для видео?,280.0,Какую категорию выбрать для видео?,"Насколько я понимаю, вы предоставили мне инфор...","[245, 90, 158, 254, 85, 278, 250, 88, 155, 92]",Вопрос: Какую категорию выбрат

In [35]:
stratified_df['ndcg'].mean()

0.15520558065159293

0.18182480949462645

0.19882406872942662

0.155142314085715

0.1834833941446053

0.15520558065159293

## Generation G-Eval

In [36]:
def clean_and_split_context(text):
    pattern = r"(Вопрос:.*?Ответ:.*?)(?=\nВопрос:|$)"

    matches = re.findall(pattern, text, re.DOTALL)

    result = [match.strip() for match in matches]
    return result


In [ ]:
for idx, row in stratified_df.iterrows():
    # display(row)
    # display(row['actual_context'])
    # break



    test_case = LLMTestCase(input=row['input'], 
                            actual_output=row['actual_response'],
                            retrieval_context=clean_and_split_context(row['actual_context']))
    
    relevance_metric.measure(test_case)
    relevance_score = relevance_metric.score
    relevance_reason = relevance_metric.reason
    
    answer_relevance_metric.measure(test_case)
    answer_relevance_score = answer_relevance_metric.score
    answer_relevance_reason = answer_relevance_metric.reason
    
    hallucination_detection_metric.measure(test_case)
    hallucination_detection_score = hallucination_detection_metric.score
    hallucination_detection_reason = hallucination_detection_metric.reason

    stratified_df.at[idx, 'relevance_score'] = relevance_score
    stratified_df.at[idx, 'relevance_reason'] = relevance_reason
    
    stratified_df.at[idx, 'answer_relevance_score'] = answer_relevance_score
    stratified_df.at[idx, 'answer_relevance_reason'] = answer_relevance_reason
    
    stratified_df.at[idx, 'hallucination_detection_score'] = hallucination_detection_score
    stratified_df.at[idx, 'hallucination_detection_reason'] = hallucination_detection_reason


In [87]:
stratified_df['hallucination_detection_score'].mean()

0.736

In [88]:
stratified_df['relevance_score'].mean()

0.128

In [89]:
stratified_df['answer_relevance_score'].mean()

0.16